In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi'] = 100

import time
from datetime import datetime, timedelta

import pylib.conv_widget as cw

In [ ]:
#Start Spark with BigDL support
from pyspark import SparkContext
import bigdl
import bigdl.util.common
sc = SparkContext.getOrCreate(conf=bigdl.util.common.create_spark_conf().setMaster("local[3]")
                              .set("spark.driver.memory","3g"))
bigdl.util.common.init_engine()

<!-- requirement: pylib/__init__.py -->
<!-- requirement: pylib/draw_nn.py -->
<!-- requirement: pylib/conv_widget.py -->
<!-- requirement: images/daisy.jpeg -->
<!-- requirement: images/horse.txt -->
<!-- requirement: images/conv_movie.gif -->
<!-- requirement: images/portal-v.png -->
<!-- requirement: images/portal-h.png -->
<!-- requirement: images/conv_weights.png -->
<!-- requirement: images/conv_layer.png -->

# Convolutional Neural Networks and Images

In the previous exercise, we flattened the MNIST images into vectors before feeding them into our model. Doing so, however, destroyed information about shapes that are present in clusters of neighboring pixels. We can preserve this two dimensional information by using **Convolutional Neural Networks** (CNNs). 

CNNs are inspired by the [visual cortex](https://en.wikipedia.org/wiki/Visual_cortex), which is responsible for image processing in animals. Neurons in the visual cortex fire in response to stimuli that activate small sub-regions of an animal's visual field. We can therefore think of these neurons as filters that detect patterns across the animal's visual field. 

We will train a CNN to find a set of filters that can detect repeated shapes in MNIST images. Convolving the image with a filter produces another image with highlighted features called a feature map. 

## Convolutions

A convolution is a mathematical operation that is performed on two functions. For continuous 1D functions, it is defined as:

$$ (f * g)(t) = \int_{-\infty}^{\infty} f(\tau)g(t-\tau) d\tau $$

For discrete functions, it is:

$$ (f * g)(n) = \sum_{m=-\infty}^{\infty} f[m] g[n-m] $$

A function that is a convolution of two other functions can be thought of as the overlap of the two original functions as one function is translated. 

![convolution](images/conv_movie.gif)

In 2D, the process is similar, except you are moving not just left-to-right, but in both directions.  Additionally, the "filters" or "kernels" that we use will tend to be small, just like the moving block in the example above.  With a center point of pixel $(m,n)$, a $3\times3$ pixel filter will do the following (with $I$ denoting pixel values and $W$ those of the filter)

$$I_{mn,new} = \sum_{i=-1}^1 \sum_{j=-1}^1 W_{ij} I_{m+i,n+j} = \sum_{i=-1}^1 (W_{i,-1} I_{m+i,n-1} + W_{i,0} I_{m+i,n} + W_{i,1} I_{m+i,n+1})$$

Play around with the widget below, which demonstrates the process of convolving a 7x7 pixel image with a 3x3 pixel filter. At each iteration, the value of one pixel in the image (denoted by the red square) is replaced by the average of the pixel values within the 3x3 filter. In convolutional neural networks, a weight is assigned to each pixel in the filter, and consequently, the "red pixel" is replaced by weighted sum of the surrounded pixels.

In [ ]:
image = np.zeros((7, 7), dtype=float)
for i in range(1, 6):
    for j in range(6-i, 6):
        image[i][j] = 1.0

img_filter = np.ones((3, 3), dtype=float)
img_filter /= img_filter.size

titles = ('Image and kernel', 'Filtered image')
convwidget = cw.ConvWidget()
convwidget.interactive_convolution_demo(image, img_filter, vmax=1, titles=titles)

#### Strides and padding

In the interactive widget, we perform the convolution at each pixel. However, we can choose to skip pixels. In BigDL, we can set the number of pixels our filter moves at each step, or the **stride**. How does the stride effect the dimension of the filtered image? Why would we want to change the stride?

Notice how the filter extends past the edge of the image while performing convolutions on some pixels. To deal with these edge cases, we typically **pad** the image or add more pixels to the perimeter of the image. This process ensures that the dimensions of the input image are the same as the dimensions of the output image. In the case of the MNIST data set, the image is padded with zeros (or white space). When might we run into problems when padding images?  What happens to the size of the filtered image if we don't?

### Example: Applying a simple edge filter on a handwritten digit

The magic behind the convolutional neural network lies in the fact that each filter learns to recognize certain patterns repeated throughout images. To build our intuition for why this is the case, we will consider two very simple filters $(1,-1)$ and $(1,-1)^T$ applied to an image consisting of only black and white pixels (with values of 1 or 0). The image consists of a black, horizontal line connected to a black, vertical line and roughly resembles a "7." See the cell below. We see from the two resulting convolved images that the filters picked out vertical and horizontal edges, respectively. 

In [ ]:
from scipy import signal as sg
ex_image = [[0, 0, 0, 0, 0],
            [1, 1, 1, 1, 0], # <---horizontal line of ones
            [0, 0, 0, 1, 0],
            [0, 0, 0, 1, 0],
            [0, 0, 0, 1, 0]]
                    # ^ vertical line of ones
plt.imshow(ex_image)
plt.grid(False)
plt.show()

    
print(sg.convolve(ex_image, [[1, -1]], "same")) #vertical edge
print()
print(sg.convolve(ex_image, [[1], [-1]], "same")) #horizontal edge


If we apply the vertical and horizontal edge filters a handwritten 7, we get the following feature maps:
<table>
    <tr>
        <td> <img src="files/images/portal-v.png" style="width: 250px;"/> </td>
        <td> <img src="files/images/portal-h.png" style="width: 250px;"/> </td>
    </tr>
</table>

### Channels and data shapes

So far we've only been looking at grayscale images, where there is a single grid of pixels.  In most situations, each pixel will either be a float between $0$ and $1$ or a `uint8` between $0$ and $256$ (8-bit values are the standard).  

What about a color image?  They're built similarly, but each pixels get not one but three values: one for red, one for green, and one for blue.  Unfortunately, how these are dealt with is a little inconsistent.  Let's look at a small ($32\times32$) image, $1024$ pixels, with three colors, for a total of $3072$ values.  The source has this as a single array.

In [ ]:
with open('images/horse.txt') as f:
    horse = np.array([int(x) for x in f.readline().split(',')], dtype=np.uint8)

In [ ]:
horse.shape

And herein lies the problem.  There are two common ways these are stored, either a $32\times32$ grid of three-element pixel vectors, for an overall $32\times32\times3$ tensor, or as three separate $32\times32$ color matrices (one for red, one for green, one for blue) for an overall $3\times32\times32$ tensor.  Most image processing software prefers the former, and BigDL (and TensorFlow) prefers the latter.

This particular example is set up in the latter format.  We'll feed BigDL three $32\times32$ images, each one representing a color channel.  This "channel" concept is actually more general, and the number of channels is the number of layers in our stack of images.  Because of this, we'll actually need to make our grayscale images explicitly 3D as well, as $1\times n \times n$ images.

In [ ]:
# Helper to display images, reshaping correctly from 3 x n x m to n x m x 3
def image_shape(img):
    red = img[0,:,:]
    green = img[1,:,:]
    blue = img[2,:,:]
    return np.stack((red,green,blue),-1)

display_horse = image_shape(horse.reshape(3,32,32))
print(display_horse)

Let's actually see this horse.

In [ ]:
plt.imshow(display_horse)
plt.grid(False)
plt.show()

If we blur out that pixelation, it becomes more clear to human eyes.

In [ ]:
plt.imshow(display_horse, interpolation='bicubic')
plt.grid(False)
plt.show()

But what about our convolutions?  How do they work with multiple channels?

They mix all the channels together, in a sense.  (This is sufficient for our purposes; image processing software can do more complicated things.)  If we have a $5\times5$ size kernel on a $3$ channel image, we don't have $25$ values, we have $75$.  Our kernel is really a block, a $3\times5\times5$ set of values, so that the output is only 2D.  Each point $(m,n)$ now looks like

$$I_{mn,new} = \sum_{i=-1}^1 \sum_{j=-1}^1 \sum_c W_{ijc} I_{m+i,n+j,c} = \sum_{i=-1}^1 \sum_{j=-1}^1 (W_{ij,r} I_{m+i,n+j,r} +  W_{ij,g} I_{m+i,n+j,g} + W_{ij,b} I_{m+i,n+j,b})$$

## Image filters

These convolutions are quite common in image processing.  They highlight specific aspects of an image, or manipulate it in a standard way.  We'll present a few common ones, but bear in mind that the ones we'll be using in our networks will _learn_ the weights to use.

We'll use a small [image of a daisy](https://commons.wikimedia.org/w/index.php?curid=45603403)
<!-- From Wikimedia commons. By Ntgr - Own work, CC BY-SA 4.0. -->

In [ ]:
from PIL import Image
im = Image.open('images/daisy.jpeg')
# Convert to grayscale, get pixel values.  Reshape array to grid, but
# im.size gives them backwards
daisy = np.array(list(im.convert('L').getdata())).reshape(im.size[1],im.size[0])

plt.imshow(daisy, cmap='gray')
plt.grid(False)

In [ ]:
# Hand apply a kernel, no padding (image out will be slightly smaller than
# image in).  This behavior is built in to BigDL and in to PIL, but we'll
# do it by hand for demonstration.
def apply_kernel(img, kernel):
    # Assume both are 2D numpy arrays
    width, height = kernel.shape
    img_w, img_h = img.shape
    
    def new_pixel(patch):
        return max(np.sum(patch*kernel),0)
    
    kernel_out = np.array([[new_pixel(img[i:i+width,j:j+height]) for j in range(0,img_h - height)] 
                     for i in range(0,img_w - width)])
    
    # Scale back down to uint8 (so 0 to 256)
    return (kernel_out / np.max(kernel_out) * 256.0).astype(np.uint8)

A blur filter:

In [ ]:
kernel = np.array([[1.0, 2.0, 1.0], [2.0, 4.0, 2.0], [1.0, 2.0, 1.0]]) / 16.0
plt.imshow(apply_kernel(daisy, kernel), cmap='gray')
plt.grid(False)

An edge detector:

In [ ]:
kernel = np.array([[-1.0, -1.0, -1.0], [-1.0, 8.0, -1.0], [-1.0, -1.0, -1.0]])
plt.imshow(apply_kernel(daisy, kernel), cmap='gray')
plt.grid(False)

### Common patterns in images

We've just talked a lot about convolutions and applied them to some images.  We've done this for a reason&mdash;we're going to build neural networks that have convolution layers, which are called Convolutional Neural Networks.  These are the current standard for dealing with images.

The reason is that these are intrinsically 2D (or 3D, depending on how you look) layers.  They look not at the pixels as just a stream of numbers, but as a grid.  In a picture horizontal and vertical should be treated roughly equally, and we expect patterns to exist in 2D.  So we'll build some convolutions to look for them.  Our edge detector above really is an intrinsically 2D behavior, and it's highlighting something important for the network to look at.  Also, we know that things that are near each other in the 2D space are likely to be similar, as real images tend not to have extremely abrupt changes the way raw numbers can.  Also, with the convolution, we're protecting ourselves a bit from translations, as it will find about the same thing with the image being shifted around.

## CNN architecture

Convolutional neural networks are comprised of a series of convolutional and max pooling layers, followed by a fully connected layer. Each of these will be explained in detail below. Let's first start off by loading in the MNIST data and making an `RDD` out of it.

In [ ]:
from bigdl.dataset import mnist
from bigdl.util.common import Sample

mnist_path = "datasets/mnist"
(train_images, train_labels) = mnist.read_data_sets(mnist_path, "train")
(test_images, test_labels) = mnist.read_data_sets(mnist_path, "test")

# Note again the "add one" to the label, since BigDL doesn't like zero
# as a label
mnist_train = sc.parallelize(zip(train_images, train_labels)).map(lambda x: Sample.from_ndarray(x[0],x[1]+1))

### Convolutional layers

The purpose of the convolutional layer is to extract features from images. In this layer, a set of filters are convolved with some input to produce activation or feature maps. Then, the ReLU operation is applied to each pixel in the feature maps, converting negative pixel values to zero. During the training process, the CNN trains filters to activate when certain features are present. 

In the example below, we choose 16 filters of size $5\times5$ with a stride of 1.

In [ ]:
from bigdl.nn import layer
model = layer.Sequential()

#This is pretty clear - turn it in to 1 layers at 28x28 - note the array around the shapes
model.add(layer.Reshape([1,28,28]))

#order: input channels (it doesn't just figure it out), output layers (i.e. # of filters), size x, size y,
#    stride x, stride y, pad x, pad y
#    strides default to 1, pads to 0.  set pad_w=-1 pad_h=-1 to get "same" behavior 
#    (i.e. in and out pixels the same)
model.add(layer.SpatialConvolution(1,16,5,5, pad_w=-1, pad_h=-1))

#activation function
model.add(layer.ReLU())

In [ ]:
#A trick to check on the shape of our output
model.forward(train_images[:3]).shape

After training, our 16 filters will look similar to those shown in the figure on the left. Red represents positive values and blue represents negative values. If we convolve an image of a seven with these filters, we will get the feature maps in the figure on the right.

<table>
    <tr>
        <td> <img src="files/images/conv_weights.png" style="width: 450px;"/> </td>
        <td> <img src="files/images/conv_layer.png" style="width: 450px;"/> </td>
    </tr>
</table>

### Max pooling 

In image classification, the approximate locations of shapes and edges are more important than their exact locations. To add this type of flexibility into our model, we can downsample our convolved images. A variety of techniques for downsampling exist, but **max pooling** is a popular one. Max pooling partitions an image and outputs the maximum value of each partition. A commonly used partition or filter size is 2x2. We set the stride of this filter to its dimension, such that pooling regions do not overlap with one another. 

What happens to the resolution of an $n \times n$ image after this type of pooling?

In [ ]:
# Max pooling layer: width, height, stride x, stride y, set pad_w=-1 and
# pad_h=-1 to get "same" behavior
model.add(layer.SpatialMaxPooling(2,2,2,2,pad_w=-1, pad_h=-1))

In [ ]:
# A trick to check on the shape of our output
model.forward(train_images[:3]).shape

### Fully connected layer

Notice that a convolution-ReLU-pooling layer operates on a single feature map (one for each convolutional filter in the previous layer), where each map represents a high-level feature in the input image. The goal of our convolutional neural network is to (1) identify each feature in the image, (2) identify combinations of these features in the image, and (3) use this information to classify the image. To this end, we eventually want to connect every neuron to one another in a **fully connected layer**. 

You can think of a fully connected layer as a layer in a basic or flat neural network. To create this layer in BigDL, we reshape our 3-dimensional output into a vector and create a linear layer, with input size given by channels * width * height ($16\times14\times14$ for us).  The output number is our decision, in this example we choose 100 as the output size. We'll then put that through an activation function, in this case `tanh`.

You can have more than one fully connected layer.  You'll tend to get better performance at the cost of more computation and more danger of overfitting.

In [ ]:
# Single-layer model
model.add(layer.Reshape([16*14*14]))
model.add(layer.Linear(16*14*14, 100))
model.add(layer.Tanh())

In [ ]:
# A trick to check on the shape of our output
model.forward(train_images[:3]).shape

### Class prediction

Finally we pass this output through another layer with output size `n_classes`, in our case 10. This step is necessary to get our predictions - we'll need our 10 $p$'s, so we'll need to route that through either a softmax or use `CrossEntropyCriterion`.

In [ ]:
model.add(layer.Linear(100,10))

### Completing the model

Our model itself is now complete, but we need to set up the infrastructure to run it - the criterion, the optimizer, and any tracking we wish to do.

We'll choose the `CrossEntropyCriterion` (which is why we didn't include a softmax layer).  Since the sigmoid and exponential functions are monotonic, we are able to say that whichever category has the largest value without the softmax would also be largest with it, so we don't actually need it to make predictions.

In [ ]:
# Set this up to 10 to run, this is set to 2 for testing reasons
n_epochs = 2 

In [ ]:
from bigdl.nn import criterion 
from bigdl.optim import optimizer

# Using Adam rather than SGD, usually performs better
fitter = optimizer.Optimizer(model=model, training_rdd=mnist_train, criterion=criterion.CrossEntropyCriterion(), 
                     optim_method=optimizer.Adam(), end_trigger=optimizer.MaxEpoch(n_epochs), 
                                batch_size=300)

# Add tracking
trainSummary = optimizer.TrainSummary("./logs", "mnist_cnn_{}".format(datetime.now().strftime("%Y%m%d-%H%M%S")))
trainSummary.set_summary_trigger("Parameters", optimizer.EveryEpoch())
fitter.set_train_summary(trainSummary)

In [ ]:
%%time
fitter.optimize()

How did we do?  Let's look at the accuracy again, as well as a few sample labeled digits.

In [ ]:
def get_accuracy(predicts, trues):
    return sum([int(predicts[i] == trues[i]) for i in range(len(predicts))]) * 1.0 / len(trues)

predictions = model.predict(mnist_train).map(lambda x: x.argmax()).collect()
get_accuracy(predictions, train_labels)

In [ ]:
#Let's get a few that it got wrong
for i in range(1,300):
    if predictions[i] != train_labels[i]:
        print(i)

In [ ]:
i = 24
print("Predict: {}, Actual: {}".format(predictions[i], train_labels[i]))
plt.imshow(train_images[i].reshape(28,28), cmap='gray')
plt.grid(False)
plt.show()

In [ ]:
#Let's also take a look at the loss function
losses = np.array(trainSummary.read_scalar('Loss'))
plt.plot(losses[:,0],losses[:,1])

## Application: Image classification

This is exactly what we've done above.  But we can do better.

### Exercise: Improve this model

Play around with the different layers.  Do you get better performance if you increase the number of convolutional filters?  The number of hidden neurons in the dense layer?

### Exercise: Bigger model


Let's see if we can improve our performance by including an additional convolutional layer in our network.

Make a new CNN with the following layers:
- 16 $5\times5$ convolutional filters, with ReLU
- $2\times2$ downsampling
- 32 $5\times5$ convolutional filters, with ReLU
- $2\times2$ downsampling
- Fully connected layer of size 256
- Output to classes

This network will run significantly slower, so start by just running it for two Epochs, to get a sense of how long it will run.  Run it for as many more epochs as you think you can get away with, at least until it starts to converge.

You'll also want to keep an eye on the CPU usage or the Spark UI to make sure the job hasn't silently crashed. If it does, you can increase the memory per worker and reduce the number of workers to trade memory for speed.

## Application: Image dimension reduction

One interesting thing we can note is that the second-to-last layer has a 100-value representation of the images of the digits.  For many purposes, we can use this in other applications as a compressed version.  This is a one-way trip, unfortunately, as we don't have a decompressor.  But this technique is used to reduce the input size of images by running them through a trained neural network like Google's Inception before using them in other applications, as this layer represents things that distinguish the photos from each other.

If we want a fuller compression method where we _can_ reconstruct the image, we need an auto-encoder.  We will be looking at these later.

## BigDL tools for image manipulation

Often with images there is a lot of pre-processing to do, particularly since most learners require uniform image shapes.  There are tools for this built in to BigDL, described [here](https://github.com/intel-analytics/BigDL/blob/master/docs/docs/ProgrammingGuide/image-support.md).  There are many useful things in this list, for example changing the color balances, but we'll just show a few simple ones

In [ ]:
from bigdl.transform.vision import image

Read in to an `ImageFrame`, the internal format BigDL uses.  We're reading a single image in, but you can get a whole directory.  Also, if you call this as `read(path, sc)` it will create RDDs instead of lists.

In [ ]:
pic = image.ImageFrame.read('images/daisy.jpeg')

Turn the raw bytes into a matrix.

In [ ]:
def bigdl_get_tensor(tens):
    #Takes in the output of image.MatToTensor
    #extracts the matrix and reshapes it to be
    #compatible with imshow
    
    #Need to reorder the colors to match what imshow expects
    return tens.get_image().__next__()[[1,2,0],:,:] / 256.0

In [ ]:
transform = image.Pipeline([image.BytesToMat(), image.MatToTensor()])

#Need to reorder the colors to match what imshow expects
mat = bigdl_get_tensor(transform(pic))
print("Initial shape: {}".format(mat.shape))
print(mat)

plt.imshow(image_shape(mat))
plt.grid(False)
plt.show()

We can make a whole pipeline of transformations to run. `AspectScale` changes the shortest dimension to the value you set.

In [ ]:
transform = image.Pipeline([image.BytesToMat(), image.AspectScale(450), image.MatToTensor()])
img = bigdl_get_tensor(transform(pic))
print("Aspect scaled shape: {}".format(img.shape))

Alternatively, you can force the image to a certain shape.  This may cause distortion, but often this is not problem.

In [ ]:
transform = image.Pipeline([image.BytesToMat(), image.Resize(400,400), image.MatToTensor()])
img = bigdl_get_tensor(transform(pic))
print("Resized shape: {}".format(img.shape))
plt.imshow(image_shape(img))
plt.grid(False)
plt.show()

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*